In [1]:
!pip install transformers datasets jsonlines folium==0.2.1

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 325 kB 44.9 MB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 37.6 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 212 kB 31.4 MB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
     |████████████████████████████████| 134 kB 37.6 MB/s 
     |████████████████████████████████| 127 kB 40.8 MB/s 
     |████████████████████████████████| 271 kB 47.3 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 46.3 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=64246ad1f7c712bb5f48452cc7f97bbbb6979d77d02223d2949b38adc2a42a98
  Stored in directory: /root/.cache/pip/wheel

# <h1><center><b>Enkóderek</b></h1></center>

## **Tokenizálás**

* Karakteralapú: túl hosszúak a szekvenciák
* Szóalapú: túl nagy a szótár
* Megoldás: **subword**
* Algoritmusok: Wordpiece, Byte Pair Encoding, Sentencepiece stb.

<br/>

<center><img src="https://drive.google.com/uc?export=view&id=1x-RgrQW3CaOu_HItCYGPQ0Wgs0a7CcFZ"/><p>Forrás: Devlin et al. (2019) </P></center>

In [2]:
from transformers import BertTokenizer

hu_tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
hu_tokenizer.save_vocabulary("hubert_vocab.txt")

Downloading:   0%|          | 0.00/266k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

('hubert_vocab.txt',)

In [5]:
!shuf -n 30 hubert_vocab.txt

Tűz
La
meghív
SD
##szülött
márka
##gazdász
##z
papr
hiányzó
interjút
konyh
##ull
kitér
munkálatok
##harang
##hárm
##KOD
##ulj
jutottak
láb
osztalék
##emlék
##idék
zsíros
[unused1792]
Én
alkalmazza
##opron
##zatos


## **Transfer learning az NLP-ben: előnyök és hátrányok**

A _transfer learning_ egy tanítási stratégia, mely a következő lépéseket feltételezi:

* Egy modell paramétereinek véletlenszerű inicializálása
* Előtanítás: a modell tanítása gradiensereszkedéssel egy olyan feladaton, melyre nagymennyiségű tanítóanyag érhető el
* Finomhangolás: az előtanított modell továbbtanítása egy célfeladaton

**Előnyök:**
* Az előtanított modell paraméterei több finomhangoláshoz is felhasználhatóak
* A finomhangolás kevesebb tanítóanyagot igényel, mint a célfeladaton történő közvetlen (előtanítás nélküli) tanítás
* A finomhangolt modell jól teljesít azon a célfeladaton, amelyen tanult a finomhangolás során 

**Hátrányok**:
* Az előtanítás sok időt és energiát vesz igénybe
* Az előtanítás eredményessége nehezen mérhető
* Az előtanított modell még nem képes kezelni a legtöbb célfeladatot
* A finomhangolás célfeladattól függő tervezést és adatokat igényel
* Egy finomhangolt modell csak egy adott célfeladatot képes ellátni

<center><img src="https://drive.google.com/uc?export=view&id=19Wl3YTCMv2B9ZdNieC3vjD0WETJ3892q"/><p>Forrás: Face Recognition Using Transfer Learning </P></center>

## **A BERT modellek: az előtanítás**

* Tanítóanyag előkészítése
* Tokenizálás
* Tanítás

<center><img src="https://drive.google.com/uc?export=view&id=1wyeMHbbwm4DtbsfEz7VURrkHBYtV-nne"/><p>Forrás: Devlin et al. (2019) </P></center>


In [2]:
!wget "https://nessie.ilab.sztaki.hu/~ndavid/Webcorpus2_text/2017_2018_0001.txt.gz"
!gzip -d "2017_2018_0001.txt.gz"

--2022-03-21 10:28:35--  https://nessie.ilab.sztaki.hu/~ndavid/Webcorpus2_text/2017_2018_0001.txt.gz
Resolving nessie.ilab.sztaki.hu (nessie.ilab.sztaki.hu)... 195.111.1.193
Connecting to nessie.ilab.sztaki.hu (nessie.ilab.sztaki.hu)|195.111.1.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2769774 (2.6M) [application/x-gzip]
Saving to: ‘2017_2018_0001.txt.gz’

2017_2018_0001.txt. 100%[===================>]   2.64M  --.-KB/s    in 0.1s    

2022-03-21 10:28:35 (19.2 MB/s) - ‘2017_2018_0001.txt.gz’ saved [2769774/2769774]

gzip: 2017_2018_0001.txt already exists; do you wish to overwrite (y or n)? y


In [ ]:
!head -n 40 "2017_2018_0001.txt"

In [4]:
!paste - - -d$"\t" <2017_2018_0001.txt >pairs.txt
!wc -l pairs.txt

44610 pairs.txt


In [2]:
from typing import Iterable, List, Tuple, Sequence, Dict, Any, Optional
from random import choices, shuffle
from itertools import tee, islice
import jsonlines

In [6]:
def get_nsp_pairs(
    lines: Iterable[str],
    num_lines: int,
    sep: str = "\t"
) -> Tuple[List[str], List[str]]:
  nopair_indices = set(choices(
      range(num_lines), k=num_lines // 2))
  pairs = []
  nopairs = []
  
  for i, sents in enumerate(map(lambda x: x.strip().split(sep), lines)):
    if len(sents) == 2:
      if "" in sents:
        nopairs.append("".join(sents))
      elif i in nopair_indices:
        for sent in sents:
          nopairs.append(sent)
      else:
        pairs.append(tuple(sents))
  
  shuffle(nopairs)
  nopairs = list(zip(*(islice(it, i, None, 2)
                       for i, it in enumerate(tee(nopairs, 2)))))
  return pairs, nopairs


def write_dataset(
    pair_sents: List[Tuple[str, str]],
    nopair_sents: List[Tuple[str, str]],
    out_path: str
) -> None:
  field1, field2 = ("sentence1", "sentence2")
  label_field = "next_sentence_label"
  with jsonlines.open(out_path, "w") as f:
    for data_source, label in zip((pair_sents, nopair_sents), (1, 0)):
      for sent1, sent2 in data_source:
        f.write({field1: sent1, field2: sent2, label_field: label})

In [7]:
with open("pairs.txt", "r", encoding="utf-8") as pairs_f:
  pair_sents, nopair_sents = get_nsp_pairs(pairs_f, 44610)

write_dataset(pair_sents, nopair_sents, out_path="dataset.jsonl")
del pair_sents, nopair_sents

In [ ]:
!shuf dataset.jsonl >dataset_shuf.jsonl
!head -n 10 dataset_shuf.jsonl

In [3]:
from transformers import (
    BatchEncoding,
    BertForPreTraining,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BertConfig
)
from datasets import Dataset, load_dataset


def tokenize_dataset(
        dataset: Dataset,
        tokenizer: BertTokenizer,
        text_col_names: Sequence[str],
        batch_size: int,
        max_seq_length: Optional[int] = None,
        label_col_name: Optional[str] = None
) -> Dataset:
    training_keys = list(tokenizer("Dummy", "input").keys())
    if label_col_name is not None:
      training_keys.append(label_col_name)
    if max_seq_length is not None:
      tokenizer.model_max_length = max_seq_length

    def tok_func(example: Dict[str, Any]) -> BatchEncoding:
      return tokenizer(
          *(example[text_col_name] for text_col_name in text_col_names),
          padding=True,
          truncation=True,
          verbose=False
      )

    dataset = dataset.map(tok_func, batched=True, batch_size=batch_size)
    dataset.set_format("torch", columns=training_keys)
    return dataset


def pre_train_bert(
    dataset: Dataset,
    tokenizer: BertTokenizer,
    label_name: str,
    output_dir: str,
    batch_size: int,
    num_epochs: int,
    learning_rate: float
) -> None:
  model = BertForPreTraining(
      config=BertConfig(vocab_size=tokenizer.vocab_size))
  data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

  training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=True,
      learning_rate=learning_rate,
      label_names=[label_name, "labels"],
      per_device_train_batch_size=batch_size,
      num_train_epochs=num_epochs
  )
  trainer = Trainer(
      model=model,
      train_dataset=dataset,
      args=training_args,
      data_collator=data_collator,
  )
  trainer.train()
  trainer.save_model(output_dir)

In [6]:
dataset = load_dataset("json", split="train", data_files=["dataset_shuf.jsonl"])
BATCH_SIZE = 8
NSP_LABEL = "next_sentence_label"
MAX_SEQ_LENGTH = 128

dataset = tokenize_dataset(
    dataset=dataset,
    tokenizer=hu_tokenizer,
    text_col_names=("sentence1", "sentence2"),
    batch_size=BATCH_SIZE,
    max_seq_length=MAX_SEQ_LENGTH,
    label_col_name=NSP_LABEL
)

Using custom data configuration default-dc8809c2804d6edd
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-dc8809c2804d6edd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


In [11]:
!rm -r sample_data && mkdir checkpoints

rm: cannot remove 'sample_data': No such file or directory


In [12]:
print(dataset[0])

{'next_sentence_label': tensor(0), 'input_ids': tensor([    2,  2142,  4575,  2714, 20488,  2325,  3576,  2079,  4721,  2313,
         3576,  2174,  2093,  8189,     3,  2155,  5719, 30742,  2066,  3218,
         4411,  2546,  6412,  3576,  5094, 24457, 26563,  2096,  4575,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
pre_train_bert(
    dataset=dataset,
    tokenizer=hu_tokenizer,
    label_name=NSP_LABEL,
    output_dir="checkpoints",
    batch_size=BATCH_SIZE,
    num_epochs=1,
    learning_rate=1e-7
)

## **Finomhangolás szentimentanalízisre**

In [16]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
from tqdm import tqdm


def fine_tune_bert(
        model: BertForSequenceClassification,
        data_loader: DataLoader,
        num_epochs: int,
        learning_rate: float = 1e-7,
        logging_freq: int = 20
) -> BertForSequenceClassification:
  """Original source: https://huggingface.co/docs/datasets/quickstart"""
  device = torch.device("cuda:0") if torch.cuda.is_available() \
      else torch.device("cpu")
  model.to(device)
  model.train()
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=learning_rate)
  for epoch in range(num_epochs):
    for i, batch in enumerate(tqdm(data_loader)):
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs[0]
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      if i % logging_freq == 0:
        print(f"Loss at step {i}, epoch {epoch}: {loss}")
  return model


In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("glue", "sst2", split="train")
dataset = tokenize_dataset(
    dataset=dataset,
    tokenizer=tokenizer,
    text_col_names=("sentence",),
    batch_size=BATCH_SIZE,
    max_seq_length=MAX_SEQ_LENGTH,
    label_col_name="label"
)
dataset = dataset.rename_column("label", "labels")
dataset = dataset.remove_columns(["sentence", "idx"])

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/8419 [00:00<?, ?ba/s]

In [14]:
print(dataset[0])

{'labels': tensor(0), 'input_ids': tensor([  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])}


In [ ]:
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
_ = fine_tune_bert(model, data_loader, num_epochs=2)

## **Sentence-BERT tanítása NLI dataseten**

* Mondatbeágyazások tanítása sziámi modellekkel
* Jól tanítható NLI-n (Natural Language Inference)
* [Dokumentáció, tananyagok](https://www.sbert.net/examples/training/nli/README.html)

<br/>

<center><img src="https://drive.google.com/uc?export=view&id=1-C8H-ExrRKcJfJQp1WaAOtCu7s58futj"/><p>Forrás: Reimers and Gurevych (2019)</p></center>


## **Cikkek, linkek**

Tokenizálás:
* Schuster et al. (2012): _Japanese and Korean Voice Search_ [Link](https://www.semanticscholar.org/paper/Japanese-and-Korean-voice-search-Schuster-Nakajima/ed6262b569c0a62c51d941228c54f34e563af022)
* Sennrich et al. (2015): _Neural Machine Translation of Rare Words with Subword Units_ [Link](https://arxiv.org/abs/1508.07909)
* Kudo and Richardson (2018): _SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing_ [Link](https://arxiv.org/abs/1808.06226)
* _Summary of the tokenizers_ (Hugging Face) [Link](https://huggingface.co/docs/transformers/tokenizer_summary)

Transfer learning:
* _Face Recognition Using Transfer Learning_ (Medium, 2020) [Link](https://medium.com/@agarwallashivam/face-recognition-using-transfer-learning-b8e36e6a15f1)

BERT:
* Devlin et al. (2019): _BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding_ [Link](https://arxiv.org/abs/1810.04805)

Sentence-BERT:
* Reimers and Gurevych (2019): _Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks_ [Link](https://arxiv.org/abs/1810.04805)

Mondatbeágyazások kiértékelése:
* Perone et al. (2018): _Evaluation of sentence embeddings in downstream and linguistic probing tasks_ [Link](https://arxiv.org/abs/1806.06259)